In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [3]:
%pip install -U tensorboard-plugin-profile


The following command must be run outside of the IPython shell:

    $ pip install -U tensorboard-plugin-profile

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(128)

In [6]:
ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [8]:
from datetime import datetime
from packaging import version

import os

In [9]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])

Epoch 1/2
469/469 [==============================] - 4s 7ms/step - loss: 0.6191 - accuracy: 0.8271 - val_loss: 0.1893 - val_accuracy: 0.9453
Epoch 2/2
469/469 [==============================] - 4s 7ms/step - loss: 0.1765 - accuracy: 0.9501 - val_loss: 0.1365 - val_accuracy: 0.9593


In [10]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [11]:
%tensorboard --logdir=logs

Reusing TensorBoard on port 6006 (pid 14571), started 0:49:12 ago. (Use '!kill 14571' to kill it.)